### Content
    0. File Path
    1. Load Vertices
    2. Load Edges
    3. Create Graph & Test
    4. GST Requirements
    5. GST Steps
    6. BT-QA main_GST Steps ( Plan )

## 4. GST Requirements
### Function: 
    call_main_GST
    (QKG_file, cornerstone_file, qtype, answer_list_file, no_GST, gdict, verbose, gt,config,con)
    
### Corresponding Parameter
    f1, f2, f4, f5, no_GST, gdict, verbose, gt1, config, context_file
        
    0. QKG_file            f1=argv[1] #input QKG
    1. cornerstone_file    f2=argv[2] #input Cornerstones 
    2. qtype               f4=argv[3] #answer match / qtype
    3. answer_list_file    f5=argv[4] #answer type /answer_list_file 
    4. no_GST              no_GST=int(argv[5]) #number of GSTs
    5. gdict               gdict
    6. verbose             verbose=int(argv[6])
    7. gt
    8. config
    9. con                 context file
    
### Corresponding Value
    0. f1 = "./files/QKG_ques-q1"  
        *** What: KG 
        *** From: Generated by Data2Graph
        *** How to use it in GST:
        
    1. f2 = "./files/QKG_cornerstones_ques-q1" 
        *** What: KG Gpickle 
        *** From: Generated by Data2Graph, cornerstones?? and them selected by ????
        *** How to use it in GST:
        
    2. f4 = "question_type.txt" 
        *** What: question type
        *** From: get_answer_types_from_questions.py
        *** How to use it in GST:used to check the answer type and select related answer
        
    3. f5 = "./files/Answer_list_ques-q1" 
        
        *** What: answer_list_file='./files/Answer_list_'+str(qid) only represent an address
        *** From: path parameter
        *** How to use it in GST: used to named answer-related address
    
    4. no_GST = 50 #Number of GSTs to be considered
        *** What: 
        *** From: config
        *** How to use it in GST:set the limited node number  
        
    5. gdict = 
            # gdict={}
            # model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  
            # word_vectors = model.wv
            # gdict=word_vectors
            
        *** What: word2vector
        *** From: gensim.models.KeyedVectors
        *** How to use it in GST: for calculate the Predicate weight(no need to used in BT-QA)

    6. verbose=int(argv[6])      
        *** What: To print intermediate statements in verbose manner make it 1, other wise 0 
        *** From: Config
        *** How to use it in GST:
    
    7. gt = set(['unknown'])
        *** What: a set of answer, the initial is 'unknown'
        *** From: 
        original from reformat_dataset_single.py
        in get_formatted_docs function,  
        fp.write('Answer\t' + 'Unknown\n')
        *** How to use it in GST:
        
    8. config stream = open("config.yml", 'r')
        for line in stream:
        if not line.startswith('#') and len(line)>1:
            line=line.split(':')
            config[line[0]]=line[1].strip()
       
    9. con = "./files/context_ques-q1"  
        *** What: context_file, not be used in GST, should be deleted

## 5. GST Steps
    - 0. load graph & corner
    - 1. initialize_queue 
        (combine the 'corner' with graph and get edges' cost between 'corner node' and 'question node' in graph. see the result T & Q)
    - 2. get GST set
    - 3. get answer type 
    - 4. trees with potential answers by filtering answer type from GST set
    - 5. answering merge (need to be changed because mention node provide more convience)
    - 6. find unique sets of answers and get result list

## 6. BT-QA main_GST Steps ( Plan )
### preprepare (libraries part)
- 0. For graph part: 
    - add node property (only for 'entity' & 'mention' nodes): weight & matched 
         - reference: "def add_node_weights" in "generate_graphs_from_triples.py"
        
    - add edges property wight & wlist 
         - reference: "def distribute_node_weights" in "generate_graphs_from_triples.py"
        
    - add type node of all 'mention' node
         - reference: "def add_type_edges" in "generate_graphs_from_triples.py"
        
    - change the id from 'number' to 'vocabulary: Node Type(doc/sent/clause/entity/mention/type)'
        
    - Structure:
   
            *Nodes:
            - ID:'vocabulary: Node Type(doc/sent/clause/entity/mention/type)'
            - Attribute:
                (u“id”, {
                ‘type’:“”
                ‘attr1’:“”
                ‘attr2’:“”
                ‘attr3’:“”
                ‘matched’:“” ##for matched questions' entities
                ‘weight’:0.0 ##for matched weight between node and questions' entities
                })
            - the weight are calculted by ‘Jaccard Coefficient’

            *Edges:
            - Node1, Node2
            - Attribute:
            (u'node1’,
             u'node2',
             {'weight': 0.0,
              'wlist': [0.0，0.0]})

    - generate corner and save it into pickle 
        reference: "generate_graphs_from triples in generate graph from triples.py"


- 1. Generate answer type from question
     - reference: get_answer_types_from_questions.py
    
### steps: ( 24 functions need to be modified, 14 of them contains graph )
- 0. load graph & corner
- 1. initialize_queue (2 def)
    (combine the 'corner' with graph and get edges' cost between 'corner node' and 'question node' in graph. see the result T & Q)
- 2. get GST set (6 def)
- 3. get answer type 
- 4. trees with potential answers by filtering answer type from GST set (4 def)
- 5. answering merge (need to be changed because Entity node provide more convience) (2 def)
- 6. find unique sets of answers and get result list (10 def)
    
    future plan: 
    I'll try to extract predicate and generate predicate node from clause to see if the precision will be enhanced or not.

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
sc.addPyFile('/usr/local/Cellar/apache-spark/2.4.1/libexec/jars/graphframes-0.7.0-spark2.4-s_2.11.jar')

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws

In [4]:
spark = SparkSession.builder \
.master("local") \
.appName("GDataFrame") \
.config("spark.sql.crossJoin.enabled", "true") \
.getOrCreate()

In [3]:
spark

In [11]:
#1. RDD -> DataFrame -> GraphFrame (code need to be change when add attr)
#2. GraphX -> GraphFrame (code no need to change when add attr) encounter problem：https://github.com/graphframes/graphframes/issues/359

## 7. Code part
### 7.1 preprepare (libraries part)
### 7.1.1 modify the id from "number" to "nodeType:Attr1:id" (1.edges, 2.nodes)

#### 1.1.1 get answer type from quetion

In [3]:
# modules
import sys
from graphframes import *
import pyspark.sql.functions as f

# library 
sys.path.append("project/library")
import get_answer_types_from_questions as gtQType
import get_all_subjects_predicates_from_questions as gtQSub
from stanfordcorenlp import StanfordCoreNLP

In [4]:
from pyspark.sql.functions import lit

In [5]:
verbose = 0

In [6]:
nlp = StanfordCoreNLP(r'/Users/jeanxu/PycharmProjects/QUEST_TerminalTest/Code/stanford-corenlp-full-2018-10-05')

In [7]:
#Prune the predicate terms from the query if the query has more than 'prune' number of terms
prune=5

In [8]:
question = 'when is national day of England'

In [9]:
#Extracting potential answer type from question
f1='question_type.txt'
gtQType.get_answer_type_main(question,f1,nlp)

In [10]:
#Extracting query terms from question.
s1='question_subject_predicate.txt'
gtQSub.get_sub_pred_ques_main(question,s1,nlp)

In [11]:
aux_list=set()
for l in open('auxiliary_verb_list.txt'):
    l=l.strip()
    aux_list.add(l.lower())

In [12]:
aux_list

{'am',
 'are',
 'be',
 'been',
 'being',
 'can',
 'could',
 'dare',
 'did',
 'do',
 'done',
 'had',
 'has',
 'have',
 'is',
 'may',
 'might',
 'must',
 'need',
 'ought',
 'shall',
 'should',
 'was',
 'were',
 'will',
 'would'}

In [13]:
# Read entities of question
s11=open(s1, 'r')
q_ent = set()
type_qent = {}
for line in s11:
    line = line.strip()
    line = line.split()
    s = line[0]
    for i in range(1, len(line) - 1):
        s += ' ' + line[i]
    q_ent.add(s.lower())
    type_qent[s.lower()] = line[len(line) - 1]
if verbose:
    print "Query terms ->", len(q_ent), q_ent
if len(q_ent) > prune:
    if verbose:
        print "Pruning.."
    p = frozenset(q_ent)
    for s in p:
        if type_qent[s] == 'P':
            q_ent.remove(s)

if verbose:
    print "Without Predicate Query terms ->", len(q_ent), q_ent

p = frozenset(q_ent)
for s in p:
    if s in aux_list and type_qent[s] == 'P':
        q_ent.remove(s)

if verbose:
    print "Without Auxiliary Query terms ->", len(q_ent), q_ent

In [14]:
type_qent

{'england': 'NE', 'is': 'P', 'national day': 'NE'}

In [15]:
q_ent

{'england', 'national day'}

## 0. File Path

In [16]:
File_verticesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Vertices/part-*"

In [17]:
File_edgesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Edges/part-*"

## 1. Load Vertices

In [18]:
verticesText0 = spark.read.csv(File_verticesTextRDD, header='false', inferSchema='false', sep='\t')
#rdd=sc.textFile(File_verticesTextRDD).map(lambda line : line.split('\t')).toDF()
#rdd.createOrReplaceTempView("vertices")
#rdd.show()

In [19]:
verticesText0.show(10)
# Document：(id, DocumentProperty(title, url, domain, tweets))
# Sentence：(id, SentenceProperty(sentenceNumber, sentenceContent, root, links.toList(lenth!=0)))
# Clause：  (id, ClauseProperty(clause,clauseType,subject,relation,argument,root,additional.toList))
# Mention： (id, MentionProperty(mention, NERtype, entity))
# Entity：  (id, EntityProperty(entity))

+--------+---+---+--------------------+-------------------+------+
|     _c0|_c1|_c2|                 _c3|                _c4|   _c5|
+--------+---+---+--------------------+-------------------+------+
|Sentence|  0|  1|Politics of Burun...|Politics of Burundi| $END$|
|Sentence|  1|  2|Executive power i...|Politics of Burundi| $END$|
|Sentence|  2|  3|Legislative power...|Politics of Burundi| $END$|
|Sentence|  3|  4|The political lan...|Politics of Burundi| $END$|
|Sentence|  4|  5|The current Presi...|Politics of Burundi| $END$|
|Sentence|  5|  6|Nkurunziza was th...|Politics of Burundi| $END$|
|Sentence|  6|  7|In November 1995,...|Politics of Burundi| $END$|
|Sentence|  7|  8|In July 1996, for...|Politics of Burundi| $END$|
|Sentence|  8|  9|He declared himse...|Politics of Burundi| $END$|
|Sentence|  9| 10|Widespread condem...|Politics of Burundi| $END$|
+--------+---+---+--------------------+-------------------+------+
only showing top 10 rows



In [ ]:
##exmaple:
# Document	5691614	Office of the United Nations High Commissioner for Human Rights			List() $END$
# Document	5691615	Gecko			List() $END$
# Document	5691616	Maeshowe			List() $END$
# Document	5691617	Kiwi			List() $END$
# Document	5691618	Iblis			List() $END$
# Document	5691619	Number			List() $END$

# Sentence	0	1	Politics of Burundi takes place in a framework of a transitional presidential representative democratic republic, whereby the President of Burundi is both head of state and head of government, and of a multi-party system.	Politics of Burundi	 $END$
# Sentence	1	2	Executive power is exercised by the government.	Politics of Burundi	 $END$

# Clause	24971	("This method", "was used", "by an estimated 38 million couples worldwide", "in 1991")	SVA	This method	was used	by an estimated 38 million couples worldwide	Coitus interruptus-3	in 1991		 $END$
# Clause	24972	("what", "was transpiring", "quickly", "spread and everybody")	UNKNOWN	what	was transpiring	quickly	First Council of Constantinople-36	spread and everybody		 $END$
# Clause	24973	("The news of what was transpiring quickly spread and everybody", "rushed", "to the church")	SVA	The news of what was transpiring quickly spread and everybody	rushed	to the church	First Council of Constantinople-36		 $END$

# Mention	24975	Freedom House	MISC	Freedom_House $END$
# Mention	24976	NGO	ORGANIZATION	Non-governmental_organization $END$
# Mention	24977	Obiang	PERSON	Teodoro_Obiang_Nguema_Mbasogo $END$

# Entity	8593089	Josephine_of_Leuchtenberg $END$
# Entity	8593090	Public_Auditorium $END$
# Entity	8593091	West_Plains,_Missouri $END$

![jupyter](./GraphExample.png)

In [20]:
verticesText1 = verticesText0.select("_c1","_c0","_c2","_c3","_c4")\
.withColumnRenamed("_c0", "nodeType").withColumnRenamed("_c1", "id")\
.withColumnRenamed("_c2", "attr1").withColumnRenamed("_c3", "attr2")\
.withColumnRenamed("_c4", "attr3")

In [21]:
verticesText1.show(10)

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
+---+--------+-----+--------------------+-------------------+
only showing top 10 rows



### 1.1 add node attributes 

In [68]:
# verticesText2=verticesText1.withColumn("did", "").withColumn("dtitle", "").withColumn("sid", "").withColumn("weight", 0.0).withColumn("matched", "")

In [77]:
# verticesText1=verticesText1.withColumn("did", lit(0)).withColumn("dtitle", lit("")).withColumn("sid", lit(0)).withColumn("weight", lit(0)).withColumn("matched", lit(""))

In [123]:
# verticesText1.show()

#### document

In [37]:
doc_verticesText=verticesText1.filter("nodeType = 'Document'")

In [38]:
doc_verticesText.show()

+----+--------+--------------------+-----+-----+
|  id|nodeType|               attr1|attr2|attr3|
+----+--------+--------------------+-----+-----+
|4526|Document|     Africa Alphabet| null| null|
|4527|Document|               Brain| null| null|
|4528|Document|     Andreas Aagesen| null| null|
|4529|Document|Australian Broadc...| null| null|
|4530|Document|The Elephant 6 Re...| null| null|
|4531|Document|Albert of Branden...| null| null|
|4532|Document|Telecommunication...| null| null|
|4533|Document|                 DMA| null| null|
|4534|Document|         Alford plea| null| null|
|4535|Document|        Claude Piron| null| null|
|4536|Document|Full disclosure (...| null| null|
|4537|Document|    Anthem of Europe| null| null|
|4538|Document|             Cytosol| null| null|
|4539|Document|            Barbagia| null| null|
|4540|Document|             Calcium| null| null|
|4541|Document|   Chomsky hierarchy| null| null|
|4542|Document| Critical philosophy| null| null|
|4543|Document|   Cu

In [39]:
#add attributes--documnets
doc_verticesText=doc_verticesText.withColumn("did", doc_verticesText.id).withColumn("dtitle", doc_verticesText.attr1).drop("attr1").drop("attr2").drop("attr3")

In [40]:
doc_verticesText.show()

+----+--------+----+--------------------+
|  id|nodeType| did|              dtitle|
+----+--------+----+--------------------+
|4526|Document|4526|     Africa Alphabet|
|4527|Document|4527|               Brain|
|4528|Document|4528|     Andreas Aagesen|
|4529|Document|4529|Australian Broadc...|
|4530|Document|4530|The Elephant 6 Re...|
|4531|Document|4531|Albert of Branden...|
|4532|Document|4532|Telecommunication...|
|4533|Document|4533|                 DMA|
|4534|Document|4534|         Alford plea|
|4535|Document|4535|        Claude Piron|
|4536|Document|4536|Full disclosure (...|
|4537|Document|4537|    Anthem of Europe|
|4538|Document|4538|             Cytosol|
|4539|Document|4539|            Barbagia|
|4540|Document|4540|             Calcium|
|4541|Document|4541|   Chomsky hierarchy|
|4542|Document|4542| Critical philosophy|
|4543|Document|4543|   Cultural movement|
|4544|Document|4544|          Braveheart|
|4545|Document|4545|                Bill|
+----+--------+----+--------------

#### sentence

In [41]:
## sentence
sen_verticesText=verticesText1.filter("nodeType = 'Sentence'")

In [42]:
sen_verticesText.show()

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
| 10|Sentence|   11|Buyoya agreed in ...|Politics of Burundi|
| 11|Sentence|   12|Nonetheless, figh...|Politics of Burundi|
| 12|Sentence|   13|In June 1998, Buy...|Politics of Burundi|
| 13|Sen

In [47]:
sen_verticesText.count()

1411580

In [43]:
# sen_verticesTextJ=sen_verticesText.join(doc_verticesText.select("did","dtitle"),sen_verticesText.attr3==doc_verticesText.dtitle, "inner")

In [44]:
# sen_verticesTextJ.show()

+-------+--------+-----+--------------------+--------------------+-------+--------------------+
|     id|nodeType|attr1|               attr2|               attr3|    did|              dtitle|
+-------+--------+-----+--------------------+--------------------+-------+--------------------+
|7282397|Sentence|    1|   A "Hello, World!"|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282398|Sentence|    2|program generally...|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282399|Sentence|    3|                   .|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282400|Sentence|    4|Because it is ver...|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282401|Sentence|    5|   A "Hello, World!"|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282402|Sentence|    6|program is tradit...|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282403|Sentence|    7|       Hello, world!|&quot;Hello, Worl...|5691624|&quot;Hello, Worl...|
|7282404|Sentence|    8|is also traditio

In [319]:
sen_verticesTextJ.count()

1410800

In [199]:
sen_edges=edgesText3.filter("label = 'contains the sentence'")

In [200]:
sen_edges.show()

+------+-------+--------------------+------+------------+-------+------------+
|   src|    dst|               label|src_id|src_nodeType| dst_id|dst_nodeType|
+------+-------+--------------------+------+------------+-------+------------+
|626404|1014942|contains the sent...|626404|    Document|1014942|    Sentence|
|626407|1015263|contains the sent...|626407|    Document|1015263|    Sentence|
|626411|1015551|contains the sent...|626411|    Document|1015551|    Sentence|
|626411|1015552|contains the sent...|626411|    Document|1015552|    Sentence|
|626411|1015610|contains the sent...|626411|    Document|1015610|    Sentence|
|626413|1015827|contains the sent...|626413|    Document|1015827|    Sentence|
|626414|1015968|contains the sent...|626414|    Document|1015968|    Sentence|
|626417|1016299|contains the sent...|626417|    Document|1016299|    Sentence|
|626417|1016518|contains the sent...|626417|    Document|1016518|    Sentence|
|626417|1016816|contains the sent...|626417|    Docu

In [201]:
sen_edgesJ=sen_edges.select("src_id","src_nodeType","dst_id","dst_nodeType")

In [202]:
sen_edgesJ.show()

+------+------------+-------+------------+
|src_id|src_nodeType| dst_id|dst_nodeType|
+------+------------+-------+------------+
|626404|    Document|1014942|    Sentence|
|626407|    Document|1015263|    Sentence|
|626411|    Document|1015551|    Sentence|
|626411|    Document|1015552|    Sentence|
|626411|    Document|1015610|    Sentence|
|626413|    Document|1015827|    Sentence|
|626414|    Document|1015968|    Sentence|
|626417|    Document|1016299|    Sentence|
|626417|    Document|1016518|    Sentence|
|626417|    Document|1016816|    Sentence|
|626418|    Document|1017045|    Sentence|
|626418|    Document|1017074|    Sentence|
|626419|    Document|1017462|    Sentence|
|626421|    Document|1017551|    Sentence|
|626422|    Document|1017716|    Sentence|
|626422|    Document|1017805|    Sentence|
|626422|    Document|1017849|    Sentence|
|626422|    Document|1018090|    Sentence|
|626422|    Document|1018100|    Sentence|
|626392|    Document|1018247|    Sentence|
+------+---

In [218]:
sen_verticesTextJ_test=verticesText1.filter("nodeType = 'Sentence'")

In [222]:
#conbine src_id
sen_edgesJ_test1=sen_edgesJ.join(doc_verticesText, sen_edgesJ.src_id==doc_verticesText.id,"inner")

In [223]:
sen_edgesJ_test1.show()

+-------+------------+-------+------------+-------+--------+-------+--------------+
| src_id|src_nodeType| dst_id|dst_nodeType|     id|nodeType|    did|        dtitle|
+-------+------------+-------+------------+-------+--------+-------+--------------+
|1019881|    Document|3245237|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245224|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245236|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245232|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245222|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245228|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245238|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245239|    Sentence|1019881|Document|1019881|Emperor Ninken|
|1019881|    Document|3245223|    Sentence|1019881|Document|1019881|Emperor 

In [224]:
sen_edgesJ_test1=sen_edgesJ_test1.drop("id").drop("nodeType")

In [225]:
#conbine dst_id
sen_verticesTextJ_test1=sen_edgesJ_test1.join(sen_verticesTextJ_test,sen_edgesJ_test1.dst_id==sen_verticesTextJ_test.id,"inner")

In [226]:
sen_verticesTextJ_test1.show()

+------+------------+-------+------------+------+--------------------+-------+--------+-----+--------------------+--------------------+
|src_id|src_nodeType| dst_id|dst_nodeType|   did|              dtitle|     id|nodeType|attr1|               attr2|               attr3|
+------+------------+-------+------------+------+--------------------+-------+--------+-----+--------------------+--------------------+
|626404|    Document|1014942|    Sentence|626404|              Genome|1014942|Sentence|  102|Both the process ...|              Genome|
|626407|    Document|1015263|    Sentence|626407|             Alcohol|1015263|Sentence|   99|In the gas phase,...|             Alcohol|
|626411|    Document|1015551|    Sentence|626411|             Cartoon|1015551|Sentence|    5|In the 19th centu...|             Cartoon|
|626411|    Document|1015552|    Sentence|626411|             Cartoon|1015552|Sentence|    6|In the early 20th...|             Cartoon|
|626411|    Document|1015610|    Sentence|626411

In [227]:
# sen_verticesTextJ_test2=sen_verticesTextJ_test1.drop("id").drop("nodeType")

In [229]:
sen_verticesTextJ_test2=sen_verticesTextJ_test1.drop("attr2").drop("attr3")

In [230]:
sen_verticesTextJ_test2.show()

+------+------------+-------+------------+------+--------------------+-------+--------+-----+
|src_id|src_nodeType| dst_id|dst_nodeType|   did|              dtitle|     id|nodeType|attr1|
+------+------------+-------+------------+------+--------------------+-------+--------+-----+
|626404|    Document|1014942|    Sentence|626404|              Genome|1014942|Sentence|  102|
|626407|    Document|1015263|    Sentence|626407|             Alcohol|1015263|Sentence|   99|
|626411|    Document|1015551|    Sentence|626411|             Cartoon|1015551|Sentence|    5|
|626411|    Document|1015552|    Sentence|626411|             Cartoon|1015552|Sentence|    6|
|626411|    Document|1015610|    Sentence|626411|             Cartoon|1015610|Sentence|   64|
|626413|    Document|1015827|    Sentence|626413|Charlton Athletic...|1015827|Sentence|  190|
|626414|    Document|1015968|    Sentence|626414|          Battleship|1015968|Sentence|  135|
|626417|    Document|1016299|    Sentence|626417|          B

In [236]:
sen_verticesText=sen_verticesTextJ_test2.drop("src_id").drop("src_nodeType").drop("dst_id").drop("dst_nodeType")

In [237]:
sen_verticesText.show()

+------+--------------------+-------+--------+-----+
|   did|              dtitle|     id|nodeType|attr1|
+------+--------------------+-------+--------+-----+
|626404|              Genome|1014942|Sentence|  102|
|626407|             Alcohol|1015263|Sentence|   99|
|626411|             Cartoon|1015551|Sentence|    5|
|626411|             Cartoon|1015552|Sentence|    6|
|626411|             Cartoon|1015610|Sentence|   64|
|626413|Charlton Athletic...|1015827|Sentence|  190|
|626414|          Battleship|1015968|Sentence|  135|
|626417|          Bangladesh|1016299|Sentence|   93|
|626417|          Bangladesh|1016518|Sentence|  312|
|626417|          Bangladesh|1016816|Sentence|  610|
|626418|               Ghost|1017045|Sentence|   40|
|626418|               Ghost|1017074|Sentence|   69|
|626419|United States For...|1017462|Sentence|  133|
|626421|         Amoxicillin|1017551|Sentence|   38|
|626422|         El Salvador|1017716|Sentence|  112|
|626422|         El Salvador|1017805|Sentence|

In [238]:
sen_verticesText=sen_verticesText.withColumnRenamed("attr1","sid")

In [239]:
sen_verticesText=sen_verticesText.select("id","nodeType","did","dtitle","sid")

In [240]:
sen_verticesText.show()

+-------+--------+------+--------------------+---+
|     id|nodeType|   did|              dtitle|sid|
+-------+--------+------+--------------------+---+
|1014942|Sentence|626404|              Genome|102|
|1015263|Sentence|626407|             Alcohol| 99|
|1015551|Sentence|626411|             Cartoon|  5|
|1015552|Sentence|626411|             Cartoon|  6|
|1015610|Sentence|626411|             Cartoon| 64|
|1015827|Sentence|626413|Charlton Athletic...|190|
|1015968|Sentence|626414|          Battleship|135|
|1016299|Sentence|626417|          Bangladesh| 93|
|1016518|Sentence|626417|          Bangladesh|312|
|1016816|Sentence|626417|          Bangladesh|610|
|1017045|Sentence|626418|               Ghost| 40|
|1017074|Sentence|626418|               Ghost| 69|
|1017462|Sentence|626419|United States For...|133|
|1017551|Sentence|626421|         Amoxicillin| 38|
|1017716|Sentence|626422|         El Salvador|112|
|1017805|Sentence|626422|         El Salvador|201|
|1017849|Sentence|626422|      

In [210]:
# sen_verticesTextJ_test1=sen_verticesTextJ_test1.drop("id").drop("nodeType")

In [213]:
# sen_verticesTextJ_test1=sen_verticesTextJ_test1.drop("attr2").withColumnRenamed("attr1","sid")

In [214]:
# sen_verticesTextJ_test1.show()

+------+------------+-------+------------+---+--------------------+
|src_id|src_nodeType| dst_id|dst_nodeType|sid|               attr3|
+------+------------+-------+------------+---+--------------------+
|626404|    Document|1014942|    Sentence|102|              Genome|
|626407|    Document|1015263|    Sentence| 99|             Alcohol|
|626411|    Document|1015551|    Sentence|  5|             Cartoon|
|626411|    Document|1015552|    Sentence|  6|             Cartoon|
|626411|    Document|1015610|    Sentence| 64|             Cartoon|
|626413|    Document|1015827|    Sentence|190|Charlton Athletic...|
|626414|    Document|1015968|    Sentence|135|          Battleship|
|626417|    Document|1016299|    Sentence| 93|          Bangladesh|
|626417|    Document|1016518|    Sentence|312|          Bangladesh|
|626417|    Document|1016816|    Sentence|610|          Bangladesh|
|626418|    Document|1017045|    Sentence| 40|               Ghost|
|626418|    Document|1017074|    Sentence| 69|  

In [46]:
# sen_verticesText=sen_verticesTextJ.withColumn("sid", sen_verticesTextJ.attr1).drop("attr1").drop("attr2").drop("attr3")

In [47]:
sen_verticesText.show()

+-------+--------+-------+--------------------+---+
|     id|nodeType|    did|              dtitle|sid|
+-------+--------+-------+--------------------+---+
|7282397|Sentence|5691624|&quot;Hello, Worl...|  1|
|7282398|Sentence|5691624|&quot;Hello, Worl...|  2|
|7282399|Sentence|5691624|&quot;Hello, Worl...|  3|
|7282400|Sentence|5691624|&quot;Hello, Worl...|  4|
|7282401|Sentence|5691624|&quot;Hello, Worl...|  5|
|7282402|Sentence|5691624|&quot;Hello, Worl...|  6|
|7282403|Sentence|5691624|&quot;Hello, Worl...|  7|
|7282404|Sentence|5691624|&quot;Hello, Worl...|  8|
|7282405|Sentence|5691624|&quot;Hello, Worl...|  9|
|7282406|Sentence|5691624|&quot;Hello, Worl...| 10|
|7282407|Sentence|5691624|&quot;Hello, Worl...| 11|
|7282408|Sentence|5691624|&quot;Hello, Worl...| 12|
|7282409|Sentence|5691624|&quot;Hello, Worl...| 13|
|7282410|Sentence|5691624|&quot;Hello, Worl...| 14|
|7282411|Sentence|5691624|&quot;Hello, Worl...| 15|
|7282412|Sentence|5691624|&quot;Hello, Worl...| 16|
|7282413|Sen

In [323]:
sen_edges.show()

+------+-------+--------------------+------+------------+-------+------------+
|   src|    dst|               label|src_id|src_nodeType| dst_id|dst_nodeType|
+------+-------+--------------------+------+------------+-------+------------+
|626404|1014942|contains the sent...|626404|    Document|1014942|    Sentence|
|626407|1015263|contains the sent...|626407|    Document|1015263|    Sentence|
|626411|1015551|contains the sent...|626411|    Document|1015551|    Sentence|
|626411|1015552|contains the sent...|626411|    Document|1015552|    Sentence|
|626411|1015610|contains the sent...|626411|    Document|1015610|    Sentence|
|626413|1015827|contains the sent...|626413|    Document|1015827|    Sentence|
|626414|1015968|contains the sent...|626414|    Document|1015968|    Sentence|
|626417|1016299|contains the sent...|626417|    Document|1016299|    Sentence|
|626417|1016518|contains the sent...|626417|    Document|1016518|    Sentence|
|626417|1016816|contains the sent...|626417|    Docu

#### clause

In [48]:
##clause 
clause_verticesText=verticesText1.filter("nodeType = 'Clause'")

In [49]:
clause_verticesText.show()

+----+--------+--------------------+-----+--------------------+
|  id|nodeType|               attr1|attr2|               attr3|
+----+--------+--------------------+-----+--------------------+
|4559|  Clause|("A significant c...|  SVA|A significant con...|
|4560|  Clause|("by the French r...|  SVO|by the French res...|
|4561|  Clause|("quinine and str...|  SVC|quinine and stryc...|
|4562|  Clause|("strychnine", "i...|  SVC|          strychnine|
|4563|  Clause|("Some writers", ...|  SVO|        Some writers|
|4564|  Clause|("those", "would ...|  SVO|               those|
|4565|  Clause|("The English Cha...|  SVC|The English Chann...|
|4566|  Clause|("Johnston", "res...|  SVO|            Johnston|
|4567|  Clause|("Tupper", "becam...|  SVO|              Tupper|
|4568|  Clause|("His father", "m...|  SVA|          His father|
|4569|  Clause|("Many physicists...|  SVA|     Many physicists|
|4570|  Clause|("of quantum mech...|  SVC|of quantum mechanics|
|4571|  Clause|("The King", "app...|  SV

In [326]:
# clause_verticesText.count()

2969676

In [50]:
clause_edges=edgesText3.filter("label = 'contains the clause'")

In [51]:
clause_edges.show()

+-------+-------+-------------------+-------+------------+-------+------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+-------------------+-------+------------+-------+------------+
|1234113| 100010|contains the clause|1234113|    Sentence| 100010|      Clause|
|3875588|1000240|contains the clause|3875588|    Sentence|1000240|      Clause|
|3650624|1000280|contains the clause|3650624|    Sentence|1000280|      Clause|
|2068926|1000665|contains the clause|2068926|    Sentence|1000665|      Clause|
| 467648|1000795|contains the clause| 467648|    Sentence|1000795|      Clause|
| 233944|1000839|contains the clause| 233944|    Sentence|1000839|      Clause|
|4022932|1000888|contains the clause|4022932|    Sentence|1000888|      Clause|
|1212133| 100140|contains the clause|1212133|    Sentence| 100140|      Clause|
| 452611|1001866|contains the clause| 452611|    Sentence|1001866|      Clause|
|1487892|1002011|contains the clause|148

In [329]:
# clause_edges.count()

2969676

In [61]:
# clause_edges_at3J=clause_edges.join(clause_verticesText.select("id","attr1"), clause_edges.dst_id==clause_verticesText.id,"inner")

In [332]:
# clause_edges_at3J.show()

+-------+-------+-------------------+-------+------------+-------+------------+-------+--------------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|     id|               attr1|
+-------+-------+-------------------+-------+------------+-------+------------+-------+--------------------+
|1234113| 100010|contains the clause|1234113|    Sentence| 100010|      Clause| 100010|("Kerensky", "ord...|
|3875588|1000240|contains the clause|3875588|    Sentence|1000240|      Clause|1000240|("to the paternal...|
|3650624|1000280|contains the clause|3650624|    Sentence|1000280|      Clause|1000280|("roughly speakin...|
|2068926|1000665|contains the clause|2068926|    Sentence|1000665|      Clause|1000665|("sympatric speci...|
| 467648|1000795|contains the clause| 467648|    Sentence|1000795|      Clause|1000795|("The growth of t...|
| 233944|1000839|contains the clause| 233944|    Sentence|1000839|      Clause|1000839|("the priests", "...|
|4022932|1000888|co

In [63]:
# clause_edges_at3J=clause_edges_at3J.drop("id")

In [65]:
# clause_edges_senJ=clause_edges_at3J.join(sen_verticesText, clause_edges_at3J.src_id==sen_verticesText.id, "inner")

In [67]:
# clause_edges_senJ=clause_edges_senJ.drop("src").drop("dst").drop("label")

In [68]:
# clause_edges_senJ.show()

+-------+------------+-------+------------+--------------------+-------+--------+------+--------------------+---+
| src_id|src_nodeType| dst_id|dst_nodeType|               attr1|     id|nodeType|   did|              dtitle|sid|
+-------+------------+-------+------------+--------------------+-------+--------+------+--------------------+---+
|1014942|    Sentence|2911087|      Clause|("Both the proces...|1014942|Sentence|626404|              Genome|102|
|1015263|    Sentence|1732278|      Clause|("alcohols", "are...|1015263|Sentence|626407|             Alcohol| 99|
|1015551|    Sentence|3236586|      Clause|("cartoon", "came...|1015551|Sentence|626411|             Cartoon|  5|
|1015552|    Sentence|3493546|      Clause|("to animated fil...|1015552|Sentence|626411|             Cartoon|  6|
|1015552|    Sentence|3493545|      Clause|("it", "began", "...|1015552|Sentence|626411|             Cartoon|  6|
|1015610|    Sentence|3327830|      Clause|("Thomson", "capi...|1015610|Sentence|626411|

In [69]:
# clause_verticesText=clause_edges_senJ.select("dst_id","dst_nodeType","attr1","did","dtitle","sid")

In [70]:
# clause_verticesText.show()

+-------+------------+--------------------+------+--------------------+---+
| dst_id|dst_nodeType|               attr1|   did|              dtitle|sid|
+-------+------------+--------------------+------+--------------------+---+
|2911087|      Clause|("Both the proces...|626404|              Genome|102|
|1732278|      Clause|("alcohols", "are...|626407|             Alcohol| 99|
|3236586|      Clause|("cartoon", "came...|626411|             Cartoon|  5|
|3493546|      Clause|("to animated fil...|626411|             Cartoon|  6|
|3493545|      Clause|("it", "began", "...|626411|             Cartoon|  6|
|3327830|      Clause|("Thomson", "capi...|626411|             Cartoon| 64|
|1230382|      Clause|("The ground-shar...|626413|Charlton Athletic...|190|
|2658479|      Clause|("the Austro-Hung...|626414|          Battleship|135|
|2658478|      Clause|("The Adriatic", ...|626414|          Battleship|135|
| 276441|      Clause|("British rule", ...|626417|          Bangladesh| 93|
| 276440|   

In [52]:
clause_verticesText_test=verticesText1.filter("nodeType = 'Clause'")

In [53]:
clause_verticesTextJ_test=clause_verticesText_test.join(clause_edges.select("src_id","src_nodeType","dst_id","dst_nodeType"), clause_verticesText_test.id==clause_edges.dst_id, "inner")

In [54]:
clause_verticesTextJ_test.show()

+-------+--------+--------------------+-----+--------------------+-------+------------+-------+------------+
|     id|nodeType|               attr1|attr2|               attr3| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+--------+--------------------+-----+--------------------+-------+------------+-------+------------+
| 100010|  Clause|("Kerensky", "ord...|  SVO|            Kerensky|1234113|    Sentence| 100010|      Clause|
|1000240|  Clause|("to the paternal...|  SVC|to the paternal h...|3875588|    Sentence|1000240|      Clause|
|1000280|  Clause|("roughly speakin...|  SVO|roughly speaking ...|3650624|    Sentence|1000280|      Clause|
|1000665|  Clause|("sympatric speci...|  SVC|sympatric speciation|2068926|    Sentence|1000665|      Clause|
|1000795|  Clause|("The growth of t...|  SVO|The growth of the...| 467648|    Sentence|1000795|      Clause|
|1000839|  Clause|("the priests", "...|  SVA|         the priests| 233944|    Sentence|1000839|      Clause|
|1000888|  Clause|(

In [55]:
clause_verticesTextJ_test=clause_verticesTextJ_test.drop("src_nodeType").drop("dst_id").drop("dst_nodeType")

In [77]:
clause_verticesTextJ_test.show()

+-------+--------+--------------------+-----+--------------------+-------+
|     id|nodeType|               attr1|attr2|               attr3| src_id|
+-------+--------+--------------------+-----+--------------------+-------+
| 100010|  Clause|("Kerensky", "ord...|  SVO|            Kerensky|1234113|
|1000240|  Clause|("to the paternal...|  SVC|to the paternal h...|3875588|
|1000280|  Clause|("roughly speakin...|  SVO|roughly speaking ...|3650624|
|1000665|  Clause|("sympatric speci...|  SVC|sympatric speciation|2068926|
|1000795|  Clause|("The growth of t...|  SVO|The growth of the...| 467648|
|1000839|  Clause|("the priests", "...|  SVA|         the priests| 233944|
|1000888|  Clause|("the absolute va...|  SVC|  the absolute value|4022932|
| 100140|  Clause|("A memorial plaq...|  SVA|   A memorial plaque|1212133|
|1001866|  Clause|("1QDan", "is", "...|  SVC|               1QDan| 452611|
|1002011|  Clause|("the use of Afri...|  SVC|the use of Africa...|1487892|
|1002185|  Clause|("he", 

In [266]:
sen_verticesText.show()

+-------+--------+-------+--------------------+---+
|     id|nodeType|    did|              dtitle|sid|
+-------+--------+-------+--------------------+---+
|7282397|Sentence|5691624|&quot;Hello, Worl...|  1|
|7282398|Sentence|5691624|&quot;Hello, Worl...|  2|
|7282399|Sentence|5691624|&quot;Hello, Worl...|  3|
|7282400|Sentence|5691624|&quot;Hello, Worl...|  4|
|7282401|Sentence|5691624|&quot;Hello, Worl...|  5|
|7282402|Sentence|5691624|&quot;Hello, Worl...|  6|
|7282403|Sentence|5691624|&quot;Hello, Worl...|  7|
|7282404|Sentence|5691624|&quot;Hello, Worl...|  8|
|7282405|Sentence|5691624|&quot;Hello, Worl...|  9|
|7282406|Sentence|5691624|&quot;Hello, Worl...| 10|
|7282407|Sentence|5691624|&quot;Hello, Worl...| 11|
|7282408|Sentence|5691624|&quot;Hello, Worl...| 12|
|7282409|Sentence|5691624|&quot;Hello, Worl...| 13|
|7282410|Sentence|5691624|&quot;Hello, Worl...| 14|
|7282411|Sentence|5691624|&quot;Hello, Worl...| 15|
|7282412|Sentence|5691624|&quot;Hello, Worl...| 16|
|7282413|Sen

In [56]:
sen_verticesTextJ=sen_verticesText.withColumnRenamed("id","senid")##important, need to keep different column name with others when "JOIN"

In [85]:
sen_verticesTextJ.show()

+-------+--------+-------+--------------------+---+
|  senid|nodeType|    did|              dtitle|sid|
+-------+--------+-------+--------------------+---+
|7282397|Sentence|5691624|&quot;Hello, Worl...|  1|
|7282398|Sentence|5691624|&quot;Hello, Worl...|  2|
|7282399|Sentence|5691624|&quot;Hello, Worl...|  3|
|7282400|Sentence|5691624|&quot;Hello, Worl...|  4|
|7282401|Sentence|5691624|&quot;Hello, Worl...|  5|
|7282402|Sentence|5691624|&quot;Hello, Worl...|  6|
|7282403|Sentence|5691624|&quot;Hello, Worl...|  7|
|7282404|Sentence|5691624|&quot;Hello, Worl...|  8|
|7282405|Sentence|5691624|&quot;Hello, Worl...|  9|
|7282406|Sentence|5691624|&quot;Hello, Worl...| 10|
|7282407|Sentence|5691624|&quot;Hello, Worl...| 11|
|7282408|Sentence|5691624|&quot;Hello, Worl...| 12|
|7282409|Sentence|5691624|&quot;Hello, Worl...| 13|
|7282410|Sentence|5691624|&quot;Hello, Worl...| 14|
|7282411|Sentence|5691624|&quot;Hello, Worl...| 15|
|7282412|Sentence|5691624|&quot;Hello, Worl...| 16|
|7282413|Sen

In [57]:
clause_verticesText_test1=clause_verticesTextJ_test.join(sen_verticesTextJ.select("senid","did","dtitle","sid"), clause_verticesTextJ_test.src_id==sen_verticesTextJ.senid, "inner")

In [58]:
clause_verticesText_test1.show()

+-------+--------+--------------------+-------+--------------------+-------+-------+------+--------------------+---+
|     id|nodeType|               attr1|  attr2|               attr3| src_id|  senid|   did|              dtitle|sid|
+-------+--------+--------------------+-------+--------------------+-------+-------+------+--------------------+---+
|2911087|  Clause|("Both the proces...|    SVA|Both the process ...|1014942|1014942|626404|              Genome|102|
|1732278|  Clause|("alcohols", "are...|    SVC|            alcohols|1015263|1015263|626407|             Alcohol| 99|
|3236586|  Clause|("cartoon", "came...|    SVO|             cartoon|1015551|1015551|626411|             Cartoon|  5|
|3493546|  Clause|("to animated fil...|    SVO|   to animated films|1015552|1015552|626411|             Cartoon|  6|
|3493545|  Clause|("it", "began", "...|    SVO|                  it|1015552|1015552|626411|             Cartoon|  6|
|3327830|  Clause|("Thomson", "capi...|UNKNOWN|             Thom

In [59]:
clause_verticesText_test2=clause_verticesText_test1.drop("attr2").drop("attr3").drop("src_id").drop("sen_id")

In [60]:
clause_verticesText_test2.show()

+-------+--------+--------------------+-------+------+--------------------+---+
|     id|nodeType|               attr1|  senid|   did|              dtitle|sid|
+-------+--------+--------------------+-------+------+--------------------+---+
|2911087|  Clause|("Both the proces...|1014942|626404|              Genome|102|
|1732278|  Clause|("alcohols", "are...|1015263|626407|             Alcohol| 99|
|3236586|  Clause|("cartoon", "came...|1015551|626411|             Cartoon|  5|
|3493546|  Clause|("to animated fil...|1015552|626411|             Cartoon|  6|
|3493545|  Clause|("it", "began", "...|1015552|626411|             Cartoon|  6|
|3327830|  Clause|("Thomson", "capi...|1015610|626411|             Cartoon| 64|
|1230382|  Clause|("The ground-shar...|1015827|626413|Charlton Athletic...|190|
|2658479|  Clause|("the Austro-Hung...|1015968|626414|          Battleship|135|
|2658478|  Clause|("The Adriatic", ...|1015968|626414|          Battleship|135|
| 276441|  Clause|("British rule", ...|1

In [61]:
clause_verticesText=clause_verticesText_test2

In [62]:
clause_verticesText.show()

+-------+--------+--------------------+-------+------+--------------------+---+
|     id|nodeType|               attr1|  senid|   did|              dtitle|sid|
+-------+--------+--------------------+-------+------+--------------------+---+
|2911087|  Clause|("Both the proces...|1014942|626404|              Genome|102|
|1732278|  Clause|("alcohols", "are...|1015263|626407|             Alcohol| 99|
|3236586|  Clause|("cartoon", "came...|1015551|626411|             Cartoon|  5|
|3493546|  Clause|("to animated fil...|1015552|626411|             Cartoon|  6|
|3493545|  Clause|("it", "began", "...|1015552|626411|             Cartoon|  6|
|3327830|  Clause|("Thomson", "capi...|1015610|626411|             Cartoon| 64|
|1230382|  Clause|("The ground-shar...|1015827|626413|Charlton Athletic...|190|
|2658479|  Clause|("the Austro-Hung...|1015968|626414|          Battleship|135|
|2658478|  Clause|("The Adriatic", ...|1015968|626414|          Battleship|135|
| 276441|  Clause|("British rule", ...|1

In [243]:
# clause_verticesText1=clause_verticesText.select("*", f.translate(f.col("attr1"), "*,", "").alias("attr2"))
clause_verticesText1 = clause_verticesText.withColumn("attr1", f.split(clause_verticesText['attr1'], ","))

In [265]:
predicate_verticesText = clause_verticesText.withColumn("predicate", f.split("attr1", ",")[1])

In [247]:
predicate_verticesText.show()

+-------+--------+--------------------+-------+------+--------------------+---+-------------------+
|     id|nodeType|               attr1|  senid|   did|              dtitle|sid|          predicate|
+-------+--------+--------------------+-------+------+--------------------+---+-------------------+
|2911087|  Clause|("Both the proces...|1014942|626404|              Genome|102|       "can result"|
|1732278|  Clause|("alcohols", "are...|1015263|626407|             Alcohol| 99|              "are"|
|3236586|  Clause|("cartoon", "came...|1015551|626411|             Cartoon|  5|             "came"|
|3493546|  Clause|("to animated fil...|1015552|626411|             Cartoon|  6|        "resembled"|
|3493545|  Clause|("it", "began", "...|1015552|626411|             Cartoon|  6|            "began"|
|3327830|  Clause|("Thomson", "capi...|1015610|626411|             Cartoon| 64|      "capitalized"|
|1230382|  Clause|("The ground-shar...|1015827|626413|Charlton Athletic...|190|              "was"|


In [266]:
predicate_verticesText1=predicate_verticesText.select("*", f.translate(f.col("predicate"), "[\"()]", "").alias("predicate1"))

In [ ]:
# df.select("A", f.regexp_replace(f.col("A"), "[\$#,]", "").alias("replaced")).show()

In [267]:
predicate_verticesText1.show()

+-------+--------+--------------------+-------+------+--------------------+---+-------------------+-----------------+
|     id|nodeType|               attr1|  senid|   did|              dtitle|sid|          predicate|       predicate1|
+-------+--------+--------------------+-------+------+--------------------+---+-------------------+-----------------+
|2911087|  Clause|("Both the proces...|1014942|626404|              Genome|102|       "can result"|       can result|
|1732278|  Clause|("alcohols", "are...|1015263|626407|             Alcohol| 99|              "are"|              are|
|3236586|  Clause|("cartoon", "came...|1015551|626411|             Cartoon|  5|             "came"|             came|
|3493546|  Clause|("to animated fil...|1015552|626411|             Cartoon|  6|        "resembled"|        resembled|
|3493545|  Clause|("it", "began", "...|1015552|626411|             Cartoon|  6|            "began"|            began|
|3327830|  Clause|("Thomson", "capi...|1015610|626411|  

In [269]:
predicate_verticesText2=predicate_verticesText1.withColumn("nodeType1",f.lit("Predicate"))

In [270]:
predicate_verticesText2.show()

+-------+--------+--------------------+-------+------+--------------------+---+-------------------+-----------------+---------+
|     id|nodeType|               attr1|  senid|   did|              dtitle|sid|          predicate|       predicate1|nodeType1|
+-------+--------+--------------------+-------+------+--------------------+---+-------------------+-----------------+---------+
|2911087|  Clause|("Both the proces...|1014942|626404|              Genome|102|       "can result"|       can result|Predicate|
|1732278|  Clause|("alcohols", "are...|1015263|626407|             Alcohol| 99|              "are"|              are|Predicate|
|3236586|  Clause|("cartoon", "came...|1015551|626411|             Cartoon|  5|             "came"|             came|Predicate|
|3493546|  Clause|("to animated fil...|1015552|626411|             Cartoon|  6|        "resembled"|        resembled|Predicate|
|3493545|  Clause|("it", "began", "...|1015552|626411|             Cartoon|  6|            "began"|     

In [273]:
predicate_verticesText=predicate_verticesText2.select("id","nodeType1","did","dtitle","sid","predicate1")

In [274]:
predicate_verticesText.show()

+-------+---------+------+--------------------+---+-----------------+
|     id|nodeType1|   did|              dtitle|sid|       predicate1|
+-------+---------+------+--------------------+---+-----------------+
|2911087|Predicate|626404|              Genome|102|       can result|
|1732278|Predicate|626407|             Alcohol| 99|              are|
|3236586|Predicate|626411|             Cartoon|  5|             came|
|3493546|Predicate|626411|             Cartoon|  6|        resembled|
|3493545|Predicate|626411|             Cartoon|  6|            began|
|3327830|Predicate|626411|             Cartoon| 64|      capitalized|
|1230382|Predicate|626413|Charlton Athletic...|190|              was|
|2658479|Predicate|626414|          Battleship|135|         remained|
|2658478|Predicate|626414|          Battleship|135|              was|
| 276441|Predicate|626417|          Bangladesh| 93|        displaced|
| 276440|Predicate|626417|          Bangladesh| 93|        broke out|
| 464234|Predicate|6

In [ ]:
##add predicate alignment

In [ ]:
#delete mention

In [63]:
# mention_verticesText=verticesText1.filter("nodeType = 'Mention'")

In [64]:
# mention_verticesText.show()

+-----+--------+--------------------+------------+--------------------+
|   id|nodeType|               attr1|       attr2|               attr3|
+-----+--------+--------------------+------------+--------------------+
|24975| Mention|       Freedom House|        MISC| Freedom_House $END$|
|24976| Mention|                 NGO|ORGANIZATION|Non-governmental_...|
|24977| Mention|              Obiang|      PERSON|Teodoro_Obiang_Ng...|
|24978| Mention|       Freedom House|        MISC| Freedom_House $END$|
|24979| Mention|                  US|    LOCATION|       --NME-- $END$|
|24980| Mention|       Freedom House|        MISC| Freedom_House $END$|
|24981| Mention|                Root|      PERSON|      Jon_Root $END$|
|24982| Mention|           Queen Ask|        MISC|       --NME-- $END$|
|24983| Mention|     Act of Congress|        MISC|United_States_Con...|
|24984| Mention|          Public Law|        MISC|Public_Law_(journ...|
|24985| Mention|              Darwin|      PERSON|Charles_Darwin

In [67]:
# mention_edges=edgesText3.filter("label = 'contains the mention'")

In [68]:
# mention_edges.show()

+-------+-------+--------------------+-------+------------+-------+------------+
|    src|    dst|               label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+--------------------+-------+------------+-------+------------+
|2073024|1007462|contains the mention|2073024|      Clause|1007462|     Mention|
|2849688|1007636|contains the mention|2849688|      Clause|1007636|     Mention|
|2568628|1007678|contains the mention|2568628|      Clause|1007678|     Mention|
| 117777|1008797|contains the mention| 117777|      Clause|1008797|     Mention|
|1759866|1009080|contains the mention|1759866|      Clause|1009080|     Mention|
|2846388|1009129|contains the mention|2846388|      Clause|1009129|     Mention|
| 564274|1009177|contains the mention| 564274|      Clause|1009177|     Mention|
| 935474|1009407|contains the mention| 935474|      Clause|1009407|     Mention|
|1878077|1009511|contains the mention|1878077|      Clause|1009511|     Mention|
|3643422|1009562|contains th

In [73]:
#delete entity & connect mention
entity_edges=edgesText3.filter("label = 'is disambiguated as'")

In [74]:
entity_edges.show()

+-------+-------+-------------------+-------+------------+-------+------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+-------------------+-------+------------+-------+------------+
|6786195|8309393|is disambiguated as|6786195|     Mention|8309393|      Entity|
|4417352|8309546|is disambiguated as|4417352|     Mention|8309546|      Entity|
|4417364|8309546|is disambiguated as|4417364|     Mention|8309546|      Entity|
|4417356|8309546|is disambiguated as|4417356|     Mention|8309546|      Entity|
|7915519|8309546|is disambiguated as|7915519|     Mention|8309546|      Entity|
|7171576|8309546|is disambiguated as|7171576|     Mention|8309546|      Entity|
|7171574|8309546|is disambiguated as|7171574|     Mention|8309546|      Entity|
|2410358|8309713|is disambiguated as|2410358|     Mention|8309713|      Entity|
|4091376|8310057|is disambiguated as|4091376|     Mention|8310057|      Entity|
|1965336|8310543|is disambiguated as|196

In [161]:
NewMentionEdges=entity_edges.groupBy("dst_id").agg(f.collect_list('src_id').alias('NewMentionEdges'))
#合并后变为array

In [162]:
NewMentionEdges=NewMentionEdges.withColumnRenamed("NewMentionEdges","Mention")

In [164]:
NewMentionEdges=NewMentionEdges.select("Mention")

In [165]:
NewMentionEdges.show()

+--------------------+
|             Mention|
+--------------------+
|           [6786195]|
|[4417352, 4417364...|
|           [2410358]|
|           [4091376]|
|[1965336, 1965344...|
|[3733707, 506663,...|
|           [2195847]|
|  [2084087, 1096926]|
|[1925341, 886511,...|
|[6556849, 7242944...|
|[5789615, 4961677...|
|[6128038, 5443558...|
|           [2438228]|
|            [120598]|
|           [7245568]|
|  [1364006, 1814055]|
|[6735411, 6745201...|
|           [2773868]|
|            [293744]|
|           [6783915]|
+--------------------+
only showing top 20 rows



In [168]:
NewMentionEdges=NewMentionEdges.withColumn("Mention1",NewMentionEdges.Mention)

In [169]:
NewMentionEdges.show()

+--------------------+--------------------+
|             Mention|            Mention1|
+--------------------+--------------------+
|           [6786195]|           [6786195]|
|[4417352, 4417364...|[4417352, 4417364...|
|           [2410358]|           [2410358]|
|           [4091376]|           [4091376]|
|[1965336, 1965344...|[1965336, 1965344...|
|[3733707, 506663,...|[3733707, 506663,...|
|           [2195847]|           [2195847]|
|  [2084087, 1096926]|  [2084087, 1096926]|
|[1925341, 886511,...|[1925341, 886511,...|
|[6556849, 7242944...|[6556849, 7242944...|
|[5789615, 4961677...|[5789615, 4961677...|
|[6128038, 5443558...|[6128038, 5443558...|
|           [2438228]|           [2438228]|
|            [120598]|            [120598]|
|           [7245568]|           [7245568]|
|  [1364006, 1814055]|  [1364006, 1814055]|
|[6735411, 6745201...|[6735411, 6745201...|
|           [2773868]|           [2773868]|
|            [293744]|            [293744]|
|           [6783915]|          

In [ ]:
# f.explode(f.split(NewMentionEdges[' NewMentionEdges'], ","))

In [124]:
# NewMentionEdges=entity_edges.select(f.concat_ws('-', entity_edges.src_id, entity_edges.dst_id).alias('NewMentionEdges'))

In [ ]:
# sample2 = sample.rdd.map(lambda x: (x.name, x.age, x.city))

In [190]:
# NewMentionEdges=NewMentionEdges
# NewMentionEdgesJ = NewMentionEdges.withColumn("Mention1", f.explode())
NewMentionEdgesJ = NewMentionEdges.withColumn("Mention1", f.explode("Mention1"))

In [183]:
NewMentionEdgesJ.show()

+--------------------+--------+
|             Mention|Mention1|
+--------------------+--------+
|           [6786195]| 6786195|
|[4417352, 4417364...| 4417352|
|[4417352, 4417364...| 4417364|
|[4417352, 4417364...| 4417356|
|[4417352, 4417364...| 7915519|
|[4417352, 4417364...| 7171576|
|[4417352, 4417364...| 7171574|
|           [2410358]| 2410358|
|           [4091376]| 4091376|
|[1965336, 1965344...| 1965336|
|[1965336, 1965344...| 1965344|
|[1965336, 1965344...| 1965346|
|[1965336, 1965344...| 1965335|
|[1965336, 1965344...| 1965349|
|[1965336, 1965344...|  401198|
|[1965336, 1965344...| 1965351|
|[3733707, 506663,...| 3733707|
|[3733707, 506663,...|  506663|
|[3733707, 506663,...|  954991|
|           [2195847]| 2195847|
+--------------------+--------+
only showing top 20 rows



In [191]:
NewMentionEdgesJ1 = NewMentionEdgesJ.withColumn("Mention", f.explode("Mention"))

In [192]:
NewMentionEdgesJ1.show()

+-------+--------+
|Mention|Mention1|
+-------+--------+
|6786195| 6786195|
|4417352| 4417352|
|4417364| 4417352|
|4417356| 4417352|
|7915519| 4417352|
|7171576| 4417352|
|7171574| 4417352|
|4417352| 4417364|
|4417364| 4417364|
|4417356| 4417364|
|7915519| 4417364|
|7171576| 4417364|
|7171574| 4417364|
|4417352| 4417356|
|4417364| 4417356|
|4417356| 4417356|
|7915519| 4417356|
|7171576| 4417356|
|7171574| 4417356|
|4417352| 7915519|
+-------+--------+
only showing top 20 rows



In [197]:
NewMentionEdgesJ2=NewMentionEdgesJ1.filter(NewMentionEdgesJ1.Mention!=NewMentionEdgesJ1.Mention1)

In [198]:
NewMentionEdgesJ2.show()

+-------+--------+
|Mention|Mention1|
+-------+--------+
|4417364| 4417352|
|4417356| 4417352|
|7915519| 4417352|
|7171576| 4417352|
|7171574| 4417352|
|4417352| 4417364|
|4417356| 4417364|
|7915519| 4417364|
|7171576| 4417364|
|7171574| 4417364|
|4417352| 4417356|
|4417364| 4417356|
|7915519| 4417356|
|7171576| 4417356|
|7171574| 4417356|
|4417352| 7915519|
|4417364| 7915519|
|4417356| 7915519|
|7171576| 7915519|
|7171574| 7915519|
+-------+--------+
only showing top 20 rows



In [75]:
# entity_edgesJ=entity_edges.select("src_id","src_nodeType","dst_id","dst_nodeType")\
# .withColumnRenamed("src_id","Jsrc_id").withColumnRenamed("src_nodeType","Jsrc_nodeType")\
# .withColumnRenamed("dst_id","Jdst_id").withColumnRenamed("dst_nodeType","Jdst_nodeType")

In [174]:
# entity_edgesJ.show()

In [ ]:
# entity_edgesJ_test=entity_edgesJ.join(entity_edgesJ,)

In [80]:
# mention_entity_edges=mention_edges.join(entity_edgesJ, mention_edges.dst_id==entity_edgesJ.Jsrc_id,"inner")

In [175]:
# mention_entity_edges.show()

In [82]:
# mention_entity_edges=mention_entity_edges.drop("dst_id").drop("dst_nodeType").drop("Jsrc_id").drop("Jsrc_nodeType")

In [176]:
# mention_entity_edges.show()

In [ ]:
##entity vertice

In [105]:
entity_verticesText=verticesText1.filter("nodeType = 'Entity'")

In [106]:
entity_verticesText.show()

+-------+--------+--------------------+-----+-----+
|     id|nodeType|               attr1|attr2|attr3|
+-------+--------+--------------------+-----+-----+
|8309308|  Entity|Gig_in_the_Sky $END$| null| null|
|8309309|  Entity|Yiannis_Ritsos $END$| null| null|
|8309310|  Entity|Doubling_the_cube...| null| null|
|8309311|  Entity|Bucerius_Law_Scho...| null| null|
|8309312|  Entity|Gresham_Professor...| null| null|
|8309313|  Entity|Danny_Barcelona $...| null| null|
|8309314|  Entity|Daallo_Airlines $...| null| null|
|8309315|  Entity|Badwater_Basin $END$| null| null|
|8309316|  Entity|Waikato_Rugby_Uni...| null| null|
|8309317|  Entity|Ev'ry_Day_of_My_L...| null| null|
|8309318|  Entity|Cała_prawda_o_pla...| null| null|
|8309319|  Entity|William_W._Thomas...| null| null|
|8309320|  Entity| Matthew_Ebden $END$| null| null|
|8309321|  Entity|   Free_Centre $END$| null| null|
|8309322|  Entity|Sacred_theology $...| null| null|
|8309323|  Entity|Maria_Sharapova $...| null| null|
|8309324|  E

In [99]:
# entity_verticesTextN = entity_verticesText.withColumn('attr1', f.regexp_replace('attr1', '$END$', '')) 

In [107]:
#delete no need string
entity_verticesText=entity_verticesText.select("*", f.translate(f.col("attr1"), " $END$", "").alias("nodeName"))

In [109]:
entity_verticesText.show()

+-------+--------+--------------------+-----+-----+--------------------+
|     id|nodeType|               attr1|attr2|attr3|            nodeName|
+-------+--------+--------------------+-----+-----+--------------------+
|8309308|  Entity|Gig_in_the_Sky $END$| null| null|      Gig_in_the_Sky|
|8309309|  Entity|Yiannis_Ritsos $END$| null| null|      Yiannis_Ritsos|
|8309310|  Entity|Doubling_the_cube...| null| null|    oubling_the_cube|
|8309311|  Entity|Bucerius_Law_Scho...| null| null| Bucerius_Law_School|
|8309312|  Entity|Gresham_Professor...| null| null|Gresham_Professor...|
|8309313|  Entity|Danny_Barcelona $...| null| null|      anny_Barcelona|
|8309314|  Entity|Daallo_Airlines $...| null| null|      aallo_Airlines|
|8309315|  Entity|Badwater_Basin $END$| null| null|      Badwater_Basin|
|8309316|  Entity|Waikato_Rugby_Uni...| null| null| Waikato_Rugby_Union|
|8309317|  Entity|Ev'ry_Day_of_My_L...| null| null|  v'ry_ay_of_My_Life|
|8309318|  Entity|Cała_prawda_o_pla...| null| null|

In [110]:
entity_verticesText=entity_verticesText.drop("attr1").drop("attr2").drop("attr3")

In [111]:
entity_verticesText.show()

+-------+--------+--------------------+
|     id|nodeType|            nodeName|
+-------+--------+--------------------+
|8309308|  Entity|      Gig_in_the_Sky|
|8309309|  Entity|      Yiannis_Ritsos|
|8309310|  Entity|    oubling_the_cube|
|8309311|  Entity| Bucerius_Law_School|
|8309312|  Entity|Gresham_Professor...|
|8309313|  Entity|      anny_Barcelona|
|8309314|  Entity|      aallo_Airlines|
|8309315|  Entity|      Badwater_Basin|
|8309316|  Entity| Waikato_Rugby_Union|
|8309317|  Entity|  v'ry_ay_of_My_Life|
|8309318|  Entity|Cała_prawda_o_pla...|
|8309319|  Entity|William_W._Thomas...|
|8309320|  Entity|        Matthew_bden|
|8309321|  Entity|         Free_Centre|
|8309322|  Entity|     Sacred_theology|
|8309323|  Entity|     Maria_Sharapova|
|8309324|  Entity|        ew_Year's_ay|
|8309325|  Entity|veryone_Says_I_Lo...|
|8309326|  Entity|         Felipe_Melo|
|8309327|  Entity|      Juvenia_Kraków|
+-------+--------+--------------------+
only showing top 20 rows



In [ ]:
#connect to mention_entity edges

In [113]:
entity_verticesTextJ=entity_verticesText.withColumnRenamed("id","Jid")

In [ ]:
mention_entity_edges.join(entity_verticesText,mention_entity_edges.dst_id==entity_verticesTextJ.id)

## 2. Load Edges

In [22]:
edgesText0 = spark.read.csv(File_edgesTextRDD)

In [23]:
edgesText0.show(10)

+------------+-------+--------------------+
|         _c0|    _c1|                 _c2|
+------------+-------+--------------------+
|Edge(5691617|7284099|contains the sent...|
|Edge(5691617|7284100|contains the sent...|
|Edge(5691617|7284101|contains the sent...|
|Edge(5691617|7284102|contains the sent...|
|Edge(5691617|7284103|contains the sent...|
|Edge(5691617|7284104|contains the sent...|
|Edge(5691617|7284105|contains the sent...|
|Edge(5691617|7284106|contains the sent...|
|Edge(5691617|7284107|contains the sent...|
|Edge(5691617|7284108|contains the sent...|
+------------+-------+--------------------+
only showing top 10 rows



In [24]:
edgesText0=edgesText0.select(f.translate(f.col("_c0"), "Edge(", "").alias("src"), "_c1", f.translate(f.col("_c2"), ")", "").alias("label"))

In [25]:
edgesText0.show()

+-------+-------+--------------------+
|    src|    _c1|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
|5691617|7284109|contains the sent...|
|5691617|7284110|contains the sent...|
|5691617|7284111|contains the sent...|
|5691617|7284112|contains the sent...|
|5691617|7284113|contains the sent...|
|5691617|7284114|contains the sent...|
|5691617|7284115|contains the sent...|
|5691617|7284116|contains the sent...|
|5691617|7284117|contains the sent...|
|5691617|7284118|contains the sent...|
+-------+-------+--------------------+
only showing top 20 rows



In [26]:
edgesText1 = edgesText0.select("*").withColumnRenamed("_c1", "dst")

In [27]:
edgesText1.show()

+-------+-------+--------------------+
|    src|    dst|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
|5691617|7284109|contains the sent...|
|5691617|7284110|contains the sent...|
|5691617|7284111|contains the sent...|
|5691617|7284112|contains the sent...|
|5691617|7284113|contains the sent...|
|5691617|7284114|contains the sent...|
|5691617|7284115|contains the sent...|
|5691617|7284116|contains the sent...|
|5691617|7284117|contains the sent...|
|5691617|7284118|contains the sent...|
+-------+-------+--------------------+
only showing top 20 rows



In [28]:
verticesText1J=verticesText1

In [32]:
# edgesText1.join(verticesText1.set_index('id'), on='src')
# edgesText1.join(verticesText1, ["src"], "outer")
# edgesText2 = edgesText1.join(verticesText1, edgesText1.src == verticesText1.id, "inner")

In [29]:
edgesText2 = edgesText1.join(verticesText1.select("id","nodeType"), edgesText1.src == verticesText1.select("id","nodeType").id, "inner")

In [30]:
edgesText2=edgesText2.withColumnRenamed("id", "src_id").withColumnRenamed("nodeType", "src_nodeType")

In [31]:
edgesText2.show(10)

+-------+-------+--------------------+-------+------------+
|    src|    dst|               label| src_id|src_nodeType|
+-------+-------+--------------------+-------+------------+
| 100010|2216143|contains the mention| 100010|      Clause|
| 100010|2216144|contains the mention| 100010|      Clause|
| 100140|3579975|contains the mention| 100140|      Clause|
| 100140|3579976|contains the mention| 100140|      Clause|
|1002011|2086735|contains the mention|1002011|      Clause|
|1002011|2086736|contains the mention|1002011|      Clause|
|1002185|3571984|contains the mention|1002185|      Clause|
|1002185|3571985|contains the mention|1002185|      Clause|
| 100227|1679175|contains the mention| 100227|      Clause|
| 100227|1679176|contains the mention| 100227|      Clause|
+-------+-------+--------------------+-------+------------+
only showing top 10 rows



In [32]:
edgesText3 = edgesText2.join(verticesText1.select("id","nodeType"), edgesText2.dst == verticesText1.select("id","nodeType").id, "inner")

In [33]:
edgesText3=edgesText3.withColumnRenamed("id", "dst_id").withColumnRenamed("nodeType", "dst_nodeType")

In [34]:
edgesText3.show(10)

+-------+-------+-------------------+-------+------------+-------+------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+-------------------+-------+------------+-------+------------+
|1234113| 100010|contains the clause|1234113|    Sentence| 100010|      Clause|
|3875588|1000240|contains the clause|3875588|    Sentence|1000240|      Clause|
|3650624|1000280|contains the clause|3650624|    Sentence|1000280|      Clause|
|2068926|1000665|contains the clause|2068926|    Sentence|1000665|      Clause|
| 467648|1000795|contains the clause| 467648|    Sentence|1000795|      Clause|
| 233944|1000839|contains the clause| 233944|    Sentence|1000839|      Clause|
|4022932|1000888|contains the clause|4022932|    Sentence|1000888|      Clause|
|1212133| 100140|contains the clause|1212133|    Sentence| 100140|      Clause|
| 452611|1001866|contains the clause| 452611|    Sentence|1001866|      Clause|
|1487892|1002011|contains the clause|148

In [35]:
edgesText3.filter("label = 'contains the sentence'").show()

+------+-------+--------------------+------+------------+-------+------------+
|   src|    dst|               label|src_id|src_nodeType| dst_id|dst_nodeType|
+------+-------+--------------------+------+------------+-------+------------+
|626404|1014942|contains the sent...|626404|    Document|1014942|    Sentence|
|626407|1015263|contains the sent...|626407|    Document|1015263|    Sentence|
|626411|1015551|contains the sent...|626411|    Document|1015551|    Sentence|
|626411|1015552|contains the sent...|626411|    Document|1015552|    Sentence|
|626411|1015610|contains the sent...|626411|    Document|1015610|    Sentence|
|626413|1015827|contains the sent...|626413|    Document|1015827|    Sentence|
|626414|1015968|contains the sent...|626414|    Document|1015968|    Sentence|
|626417|1016299|contains the sent...|626417|    Document|1016299|    Sentence|
|626417|1016518|contains the sent...|626417|    Document|1016518|    Sentence|
|626417|1016816|contains the sent...|626417|    Docu

In [36]:
verticesText1.filter("nodeType = 'Clause'").show()##remember here the  attr3 is not complete, however, this data will not be used.

+----+--------+--------------------+-----+--------------------+
|  id|nodeType|               attr1|attr2|               attr3|
+----+--------+--------------------+-----+--------------------+
|4559|  Clause|("A significant c...|  SVA|A significant con...|
|4560|  Clause|("by the French r...|  SVO|by the French res...|
|4561|  Clause|("quinine and str...|  SVC|quinine and stryc...|
|4562|  Clause|("strychnine", "i...|  SVC|          strychnine|
|4563|  Clause|("Some writers", ...|  SVO|        Some writers|
|4564|  Clause|("those", "would ...|  SVO|               those|
|4565|  Clause|("The English Cha...|  SVC|The English Chann...|
|4566|  Clause|("Johnston", "res...|  SVO|            Johnston|
|4567|  Clause|("Tupper", "becam...|  SVO|              Tupper|
|4568|  Clause|("His father", "m...|  SVA|          His father|
|4569|  Clause|("Many physicists...|  SVA|     Many physicists|
|4570|  Clause|("of quantum mech...|  SVC|of quantum mechanics|
|4571|  Clause|("The King", "app...|  SV

## 3. Create Graph & Test

In [28]:
Graph = GraphFrame(verticesText1, edgesText1)

In [29]:
Graph.edges.show(10)

+-------+-------+--------------------+
|    src|    dst|               label|
+-------+-------+--------------------+
|5691617|7284099|contains the sent...|
|5691617|7284100|contains the sent...|
|5691617|7284101|contains the sent...|
|5691617|7284102|contains the sent...|
|5691617|7284103|contains the sent...|
|5691617|7284104|contains the sent...|
|5691617|7284105|contains the sent...|
|5691617|7284106|contains the sent...|
|5691617|7284107|contains the sent...|
|5691617|7284108|contains the sent...|
+-------+-------+--------------------+
only showing top 10 rows



In [30]:
Graph.vertices.show(10)

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
+---+--------+-----+--------------------+-------------------+
only showing top 10 rows



In [31]:
Graph.vertices.count()

8593092

In [32]:
Graph.edges.count()

11607488

In [33]:
# Example 1: bfs Algorithm
filteredPaths = Graph.bfs(
  fromExpr = "id = '5691617'",
  toExpr = "id = '7284108'",
  maxPathLength = 2)

In [34]:
# display(filteredPaths)
filteredPaths.show()

+--------------------+--------------------+--------------------+
|                from|                  e0|                  to|
+--------------------+--------------------+--------------------+
|[5691617, Documen...|[5691617, 7284108...|[7284108, Sentenc...|
+--------------------+--------------------+--------------------+

